In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import wave
import librosa
from pathlib import Path
from IPython.display import display, Audio

import beatbrain

import torch
import torch.nn as nn

In [11]:
AUDIO_PATH = "../data/shetti/wavs-22050-split/11 - The Vulture_12.wav"
audio, sr = librosa.load(AUDIO_PATH)
display(Audio(audio, rate=sr))

In [ ]:
class MelSpec(nn.Module):
    # TODO: Copy params and forward pass from MelGan
    def __init__(self, *args, **kwargs):
        pass
    def forward(self, audio):
        """
        Audio to mel spectrogram
        """
        pass
    def inverse(self, spec):
        """
        Best-attempt spectrogram inversion
        """
        pass